<a href="https://colab.research.google.com/github/cehaletx/pub/blob/main/last_N_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a notebook to work through ingest rates for an elastic cluster.

It will connect to the cloud id using the user and pass which it will prompt for then calculate the average doc size and the number of docs for the last
N days.

First step let's import the right libraries.

In [ ]:
!pip install -q elasticsearch
import sys
from datetime import datetime
from datetime import timedelta
import re
import time
import json
from elasticsearch import Elasticsearch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

# Let's define some variables, the number of days and the index patterns.

In [ ]:
#--------------------------------------------------------------------------------
# not doing any checking but let's get the command line arg. expecting an
# integer which is number of days of ingest to calculate
#--------------------------------------------------------------------------------
#last_n_days = int( sys.argv[1] )
last_n_days = int (7)
index_pattern="*"

# Let's collect the elastic connection credentials


In [ ]:
#ELASTIC_CLOUD_ID=input("input your cloud id ")
ELASTIC_CLOUD_ID="demo-cluster:dXMtZWFzdC0xLmF3cy5mb3VuZC5pbzo0NDMkYThhNGM3NDVlNTU4NGU4OGFmMWQ1OGFkY2E5NTBkN2IkMmIwYTk2ZjY4MDE0NGFjNzk4Y2NiMDA4NjUyMDc0YWE="
default = "elastic"
ELASTIC_USERNAME = "elastic"
#ELASTIC_USERNAME=input("input your elastic user [%s] "%default + chr(8)*4)
#if not ELASTIC_USERNAME:
#   ELASTIC_USERNAME = default
#ELASTIC_PASSWORD=input("input your elastic password ")
ELASTIC_PASSWORD="3QndbuYUwKiyN5EyxZ9IdByN"

In [ ]:
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    timeout=60,
    basic_auth=(ELASTIC_USERNAME, ELASTIC_PASSWORD)
)

# Setup the data class, inititalize some arrays, and define a function to calculate document size

In [ ]:
class doubleQuoteDict(dict):
  def __str__(self):
    return json.dumps(self)
  def __repr__(self):
    return json.dumps(self)

# Initialize lists to store data
periods = []
document_counts = []

# Function to query Elasticsearch and get document count for a given time range
def get_document_count(index_pattern, start_time, end_time):
    range_query = {
      "range": {
        "@timestamp": {
          "gte": start_time,
          "lt": end_time
        }
      }
    }

    result = es.search(index=index_pattern, query=doubleQuoteDict(range_query), params={"track_total_hits":"true"} )
    return result['hits']['total']['value']

# Now let's get the average doc size.

In [ ]:
# before looking at the previous N days, let's inspect cat indices index_pattern for a doc count, and avg size
#
resp = es.cat.indices( index=index_pattern, format="json",bytes="b" )

temp_total_docs = 0
temp_total_pri = 0
new_avg_doc = 0
for each_index in resp:
  if( int( each_index["pri.store.size"] ) != 0 ):
    temp_total_docs+= int( each_index["docs.count"] )
    temp_total_pri+= int( each_index["pri.store.size"] )
new_avg_doc = temp_total_pri / temp_total_docs
print( "total docs = ", temp_total_docs)
print( "total pri size = ", temp_total_pri)
print ( "avg doc size = ", new_avg_doc )

Now let's go through and look at the number of docs per day

In [ ]:
daily=[]
periods=[]
while( last_n_days >= 1 ):
  date_n1_days_ago = datetime.now() - timedelta( days=(last_n_days+1) )
  date_n_days_ago = datetime.now() - timedelta( days=last_n_days )
  print ( "start = " + str(date_n1_days_ago) )
  print ( "end = " + str(date_n_days_ago) )
  print ( "index pattern - " + index_pattern )
  doc_count = get_document_count(index_pattern, date_n1_days_ago, date_n_days_ago)
  print("%d docs " % doc_count)
  gigs = (doc_count * new_avg_doc) / 1024 / 1024 / 1024
  daily.append( gigs )
  periods.append( date_n_days_ago.strftime("%m-%d") )
  mb = (doc_count * new_avg_doc) / 1024 / 1024
  print("in gigs = " + str(gigs) + ", or in mb = " + str(mb) + ", or in bytes = "+ str(doc_count*new_avg_doc)  )
  last_n_days = last_n_days - 1

plt.bar( periods, daily, label="Daily Total", color='b')
plt.xlabel("Day")
plt.ylabel("Volume")
plt.title("Ingest Volume Per day in GB")
plt.legend()
plt.show()